### Deteccion de vehiculo en área de interes y prediccion con etiqueta

In [ ]:
import cv2
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

# Load car classifier
car_classifier = cv2.CascadeClassifier('../datasets/cars.xml')

# Load video file
cap = cv2.VideoCapture('videos/mer1_mod.mp4')

# Set video FPS to 20
cap.set(cv2.CAP_PROP_FPS, 1)


#load model
modelt = load_model("../nn_model_vgg_19.h5")
names = ['bus', 'car', 'truck']

# Loop over the frames from the video
while cap.isOpened():
    # Read the next frame from the video
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect cars in the frame
  
 
    cars = car_classifier.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(50, 50))

    
    # Draw bounding boxes and labels around the vehicles 
    
    for (x,y,w,h) in cars:
        #texto = "Prueba"
        posicion_etiqueta = (x , y - 10)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        
        #hacer la clasificacion
        # Extract image of the detected car
        
        vehicle_image_to_classifier = frame[y:y+h, x:x+w]#Imagen a clasificar

        #clasificar la seleccion:
        imaget=cv2.resize(vehicle_image_to_classifier, (224, 224), interpolation = cv2.INTER_AREA)
        
        xt = np.asarray(imaget)
        xt = preprocess_input(xt)
        xt = np.expand_dims(xt,axis=0)
        preds = modelt.predict(xt)

        #Colocar etiqueta
        cv2.putText(frame, names[np.argmax(preds)], posicion_etiqueta, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    # Wait for user input to move to the next frame
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()